# VacationPy
---

## An analysis in pursuit of the perfect vacation

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/antonettekg/Desktop/Challenges/python-api/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port hawkesbury,45.6169,-61.3485,30.18,67,20,13.80,CA,1678052615
1,1,bluff,-46.6000,168.3333,53.80,69,37,6.55,NZ,1678052615
2,2,basco,20.4487,121.9702,71.11,64,100,21.63,PH,1678052616
3,3,sinegorye,62.0788,150.4770,-20.43,92,91,0.65,RU,1678052617
4,4,lebu,-37.6167,-73.6500,63.39,85,5,9.57,CL,1678052617


### Display Map of Relative Humidity by City

In [17]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    frame_width = 900,
    frame_height = 600,
    xlabel = "Longitude",
    ylabel = "Latitude",
    tiles = "OSM",
    color = "City",
    alpha = 1,
)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [8]:
# Narrow cities that fit criteria for ideal weather based on temperature and wind speed

city_ideal_df = city_data_df.loc[(city_data_df['Max Temp'] >= 25) & (city_data_df['Max Temp'] <= 30) & 
                                  (city_data_df['Wind Speed'] <= 4.5) & (city_data_df['Cloudiness'] <= 20)]

# Drop any rows with null values
city_ideal_df = city_data_df.dropna(how='any')

# Display new df

city_ideal_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port hawkesbury,45.6169,-61.3485,30.18,67,20,13.80,CA,1678052615
1,1,bluff,-46.6000,168.3333,53.80,69,37,6.55,NZ,1678052615
2,2,basco,20.4487,121.9702,71.11,64,100,21.63,PH,1678052616
3,3,sinegorye,62.0788,150.4770,-20.43,92,91,0.65,RU,1678052617
4,4,lebu,-37.6167,-73.6500,63.39,85,5,9.57,CL,1678052617


In [9]:
# Create new DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal_df.loc[:, [("City"), ("Country"), ("Lat"), ("Lng"), ("Humidity"),]]


# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity
0,port hawkesbury,CA,45.6169,-61.3485,67
1,bluff,NZ,-46.6000,168.3333,69
2,basco,PH,20.4487,121.9702,64
3,sinegorye,RU,62.0788,150.4770,92
4,lebu,CL,-37.6167,-73.6500,85
5,saint george,US,37.1041,-113.5841,30
6,priargunsk,RU,50.3691,119.1012,96
7,warrnambool,AU,-38.3833,142.4833,77
8,faanui,PF,-16.4833,-151.7500,72
9,mandalgovi,MN,45.7625,106.2708,32


In [10]:
# Set parameters to search Geoapify for a hotel within 10K meters of each city.
radius = 10000
params = {"q": "Hotel_Name", 
          "categories":"accommodation.hotel", 
    "radius": radius,
    "apiKey": geoapify_key}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iloc[:20].iterrows():   
    # get latitude, longitude from the existing DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

   # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # JSONify the response
    name_address = response.json()
    
    # Take the first hotel from results and store name in hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel_Name"] = name_address["features"][0]["properties"]["name"]
    # If no hotel is found, set the hotel name as "No hotel found".
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel_Name"] = "No hotel found"
        
    #Log the search results as it is ongoing
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel_Name']}")
    
    
print("Hotel Search Complete")


Starting hotel search
port hawkesbury - nearest hotel: Hearthstone Inn Port Hawkesbury
bluff - nearest hotel: No hotel found
basco - nearest hotel: Tawsen Hotel
sinegorye - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
priargunsk - nearest hotel: No hotel found
warrnambool - nearest hotel: Central Court
faanui - nearest hotel: Oa Oa Lodge
mandalgovi - nearest hotel: АЛТАНГОВЬ Hotel
laguna - nearest hotel: Holiday Inn Express & Suites
qaanaaq - nearest hotel: Hotel Qaanaaq
teguise - nearest hotel: Finca Las Laderas
punta arenas - nearest hotel: Hotel Lacolet
moose factory - nearest hotel: Cree Village Ecolodge
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
thompson - nearest hotel: Thompson Inn
champerico - nearest hotel: Hotel y Restaurante El Submarino
ergani - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
Hotel Search Complete


In [19]:
# Display sample dataframe of retrieved data
hotel_df.head(15)

,City,Country,Lat,Lng,Humidity,Hotel_Name
0,port hawkesbury,CA,45.6169,-61.3485,67,Hearthstone Inn Port Hawkesbury
1,bluff,NZ,-46.6000,168.3333,69,No hotel found
2,basco,PH,20.4487,121.9702,64,Tawsen Hotel
3,sinegorye,RU,62.0788,150.4770,92,No hotel found
4,lebu,CL,-37.6167,-73.6500,85,No hotel found
5,saint george,US,37.1041,-113.5841,30,"The Advenire, an Autograph Collection Hotel"
6,priargunsk,RU,50.3691,119.1012,96,No hotel found
7,warrnambool,AU,-38.3833,142.4833,77,Central Court
8,faanui,PF,-16.4833,-151.7500,72,Oa Oa Lodge
9,mandalgovi,MN,45.7625,106.2708,32,АЛТАНГОВЬ Hotel


In [18]:
# Configure the map plot with additional criteria of hotel & country
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    xlabel = "Longitude",
    ylabel = "Latitude",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel_Name", "Country"]
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel_Name,Country)